# AIVLE스쿨 2차 미니프로젝트: 신규 아파트 주차 수요 예측

<img src = "https://github.com/Jangrae/img/blob/master/parking.png?raw=true" width=800, align="left"/>

# 단계 1: 데이터 전처리

## [미션]

단지별 등록 차량 수를 예측하기에 적합한 형태로 데이터 전처리를 수행합니다.

1) 필요한 변수를 추가하고 불필요한 변수를 제거합니다.
2) 단지별 데이터와 상세 데이터를 분리합니다.
3) 상세 데이터를 단지별로 집계합니다.
    - 단지별 총면적 집계
    - 전용면적구간 집계 (피벗형태)
    - 단지별 임대보증금, 임대료 평균 집계
4) 단지별 데이터와 집계 데이터를 하나로 합칩니다.
5) 변수 추가 (옵션)
    - 등록 차량수를 예측하기 위해 필요한 변수를 추가합니다.

## 1. 환경설정

### (1) 로컬 수행(Anaconda)

- project 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

In [393]:
# 기본 경로
path = ''

### (2) 구글 콜랩 수행

- 구글 콜랩을 사용중이면 구글 드라이브를 연결합니다.

In [395]:
# 구글 드라이브 연결, 패스 지정
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    path = '/content/drive/MyDrive/project/'

### (3) 한글 폰트 표시용 라이브러리 설치

In [397]:
# 한글 표시를 위한 라이브러리 설치
!pip install koreanize_matplotlib -q

### (4) 라이브러리 불러오기

In [399]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns

import joblib
import warnings

warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format='retina'

### (5) 데이터 불러오기

- 학습용 데이터(train.xlsx)를 읽어옵니다.
- 읽어온 데이터를 apart 데이터프레임으로 선언합니다.
- 평가용 데이터(test.xlsx) 파일은 모델 완성 후 사용할 미래의 데이터입니다.

#### 1) 데이터 불러오기

In [402]:
# 파일 불러오기
apart = pd.read_excel(path+'train.xlsx')

#### 2) 기본 정보 조회

In [404]:
apart.shape

(1157, 15)

In [405]:
apart.head()

,단지코드,단지명,총세대수,전용면적별세대수,지역,준공일자,건물형태,난방방식,승강기설치여부,단지내주차면수,전용면적,공용면적,임대보증금,임대료,실차량수
0,C0001,엘에이치 서초4단지,78,35,서울,20131204,계단식,개별가스난방,전체동 설치,120,51.89,19.2603,50758000,620370,109
1,C0001,엘에이치 서초4단지,78,43,서울,20131204,계단식,개별가스난방,전체동 설치,120,59.93,22.2446,63166000,665490,109
2,C0002,LH삼성아파트,35,26,서울,20130801,복도식,개별가스난방,전체동 설치,47,27.75,16.5375,63062000,458640,35
3,C0002,LH삼성아파트,35,9,서울,20130801,복도식,개별가스난방,전체동 설치,47,29.08,17.3302,63062000,481560,35
4,C0003,강남LH8단지,88,7,서울,20131023,계단식,개별가스난방,전체동 설치,106,59.47,21.9462,72190000,586540,88


In [406]:
apart.tail()

,단지코드,단지명,총세대수,전용면적별세대수,지역,준공일자,건물형태,난방방식,승강기설치여부,단지내주차면수,전용면적,공용면적,임대보증금,임대료,실차량수
1152,C0356,덕유마을 주공1단지,956,956,경기,19941130,복도식,지역가스난방,전체동 설치,202,26.37,12.7500,9931000,134540,243
1153,C0358,화천신읍(공공실버) 영구임대,120,66,강원,20200110,복도식,NaN,전체동 설치,40,24.83,15.1557,2129000,42350,47
1154,C0358,화천신읍(공공실버) 영구임대,120,54,강원,20200110,복도식,NaN,전체동 설치,40,33.84,20.6553,2902000,57730,47
1155,C0359,영천야사4,447,149,대구경북,19940615,복도식,중앙유류난방,전체동 설치,89,26.37,13.3800,7134000,118880,78
1156,C0359,영천야사4,447,298,대구경북,19940615,복도식,중앙유류난방,전체동 설치,89,31.32,13.8500,8122000,131140,78


In [407]:
apart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1157 entries, 0 to 1156
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   단지코드      1157 non-null   object 
 1   단지명       1157 non-null   object 
 2   총세대수      1157 non-null   int64  
 3   전용면적별세대수  1157 non-null   int64  
 4   지역        1157 non-null   object 
 5   준공일자      1157 non-null   int64  
 6   건물형태      1135 non-null   object 
 7   난방방식      1082 non-null   object 
 8   승강기설치여부   1059 non-null   object 
 9   단지내주차면수   1157 non-null   int64  
 10  전용면적      1157 non-null   float64
 11  공용면적      1157 non-null   float64
 12  임대보증금     1157 non-null   int64  
 13  임대료       1157 non-null   int64  
 14  실차량수      1157 non-null   int64  
dtypes: float64(2), int64(7), object(6)
memory usage: 135.7+ KB


## 2. 데이터 전처리 ①

- 결측치 존재 여부를 확인하고 적절히 처리합니다.
- 필요한 변수를 추가하고, 불필요한 변수를 제거합니다.

### (1) 결측치 처리

- 결측치가 있는 지 확인합니다.

In [410]:
apart.isna().sum()

단지코드         0
단지명          0
총세대수         0
전용면적별세대수     0
지역           0
준공일자         0
건물형태        22
난방방식        75
승강기설치여부     98
단지내주차면수      0
전용면적         0
공용면적         0
임대보증금        0
임대료          0
실차량수         0
dtype: int64

- 결측치는 적절한 값으로 채웁니다.
- 예를 들어 범주형 변수인 경우는 각 변수의 최빈값으로 채울 수 있습니다.

In [412]:
apart['건물형태'] = apart['건물형태'].fillna(apart['건물형태'].mode()[0])
apart['난방방식'] = apart['난방방식'].fillna(apart['난방방식'].mode()[0])
apart['승강기설치여부'] = apart['승강기설치여부'].fillna(apart['승강기설치여부'].mode()[0])

### (2) 변수 추가

- '준공일자' 변수 값 앞 4 자리를 갖는 int 형 변수 '준공연도'를 추가합니다.
- 총면적 = (전용면적 + 공용면적) * 전용면적별세대수 공식에 따른'총면적' 변수를 추가합니다.

In [414]:
apart['준공연도'] = apart['준공일자']//10000
apart['총면적'] = (apart['전용면적'] + apart['공용면적']) * apart['전용면적별세대수']

In [415]:
apart.head()

,단지코드,단지명,총세대수,전용면적별세대수,지역,준공일자,건물형태,난방방식,승강기설치여부,단지내주차면수,전용면적,공용면적,임대보증금,임대료,실차량수,준공연도,총면적
0,C0001,엘에이치 서초4단지,78,35,서울,20131204,계단식,개별가스난방,전체동 설치,120,51.89,19.2603,50758000,620370,109,2013,2490.2605
1,C0001,엘에이치 서초4단지,78,43,서울,20131204,계단식,개별가스난방,전체동 설치,120,59.93,22.2446,63166000,665490,109,2013,3533.5078
2,C0002,LH삼성아파트,35,26,서울,20130801,복도식,개별가스난방,전체동 설치,47,27.75,16.5375,63062000,458640,35,2013,1151.4750
3,C0002,LH삼성아파트,35,9,서울,20130801,복도식,개별가스난방,전체동 설치,47,29.08,17.3302,63062000,481560,35,2013,417.6918
4,C0003,강남LH8단지,88,7,서울,20131023,계단식,개별가스난방,전체동 설치,106,59.47,21.9462,72190000,586540,88,2013,569.9134


### (3) 불필요한 변수 제거

- '단지명' 변수는 단일값을 가지므로 제거합니다.
- '단지내주차면수' 변숫값을 기반으로 등록 차량수를 예측하는 것은 의미가 없으니, '단지내주차면수' 변수를 제거합니다.
- '준공연도' 변수를 추가했으니 '준공일자' 변수를 제거합니다.

In [417]:
del_cols = ['단지명','단지내주차면수','준공일자']
apart.drop(columns=del_cols, inplace=True)

In [418]:
apart.head()

,단지코드,총세대수,전용면적별세대수,지역,건물형태,난방방식,승강기설치여부,전용면적,공용면적,임대보증금,임대료,실차량수,준공연도,총면적
0,C0001,78,35,서울,계단식,개별가스난방,전체동 설치,51.89,19.2603,50758000,620370,109,2013,2490.2605
1,C0001,78,43,서울,계단식,개별가스난방,전체동 설치,59.93,22.2446,63166000,665490,109,2013,3533.5078
2,C0002,35,26,서울,복도식,개별가스난방,전체동 설치,27.75,16.5375,63062000,458640,35,2013,1151.4750
3,C0002,35,9,서울,복도식,개별가스난방,전체동 설치,29.08,17.3302,63062000,481560,35,2013,417.6918
4,C0003,88,7,서울,계단식,개별가스난방,전체동 설치,59.47,21.9462,72190000,586540,88,2013,569.9134


## 3. 데이터 전처리 ②

- 단지별 데이터와 상세 데이터로 분리합니다.
- 상세 데이터를 3가지 형태로 집계합니다.
- 단지별 데이터와 상세 데이터 집계 결과를 조인(Merge) 합니다.

### (1) 데이터 분리

- 단지별 데이터를 갖는 data01 데이터프레임을 선언합니다.
- 상세 데이터를 갖는 data02 데이터프레임을 선언합니다.

#### 1) 단지별 데이터 분리

- 다음 열을 갖는 data01 데이터프레임으로 선언합니다.
    - '단지코드', '총세대수', '지역', '준공연도', '건물형태', '난방방식', '승강기설치여부', '실차량수'
- data01 데이터프레임의 중복행을 제거합니다.
- 인덱스를 초기화 합니다. (단, 기존 인덱스 제거)
- 중복행 제거 여부를 필히 확인합니다.

In [422]:
data01 = apart.loc[:,['단지코드', '총세대수', '지역', '준공연도', '건물형태', '난방방식', '승강기설치여부', '실차량수']]

In [423]:
data01.drop_duplicates(ignore_index= True, inplace = True)
data01.head()

,단지코드,총세대수,지역,준공연도,건물형태,난방방식,승강기설치여부,실차량수
0,C0001,78,서울,2013,계단식,개별가스난방,전체동 설치,109
1,C0002,35,서울,2013,복도식,개별가스난방,전체동 설치,35
2,C0003,88,서울,2013,계단식,개별가스난방,전체동 설치,88
3,C0004,477,서울,2014,복도식,지역난방,전체동 설치,943
4,C0006,15,서울,2013,복도식,개별가스난방,전체동 설치,21


#### 2) 상세 데이터 분리
    
- 다음 열을 갖는 data02 데이터프레임으로 선언합니다.
    - '단지코드', '총면적', '전용면적별세대수', '전용면적', '공용면적', '임대보증금', '임대료'

In [425]:
data02 = apart.loc[:,['단지코드', '총면적', '전용면적별세대수', '전용면적', '공용면적', '임대보증금', '임대료']]
data02.head()

,단지코드,총면적,전용면적별세대수,전용면적,공용면적,임대보증금,임대료
0,C0001,2490.2605,35,51.89,19.2603,50758000,620370
1,C0001,3533.5078,43,59.93,22.2446,63166000,665490
2,C0002,1151.4750,26,27.75,16.5375,63062000,458640
3,C0002,417.6918,9,29.08,17.3302,63062000,481560
4,C0003,569.9134,7,59.47,21.9462,72190000,586540


### (2) 상세 데이터 집계

- 앞에서 선언한 data02 데이터프레임을 대상으로 다음 3가지 형태로 집계합니다.
    - 단지코드별 총면적 합을 집계합니다.
    - 전용면적을 의미있는 구간으로 나누어 피벗 형태로 집계합니다.
    - 단지코드별 임대보증금, 임대료 평균을 집계합니다

#### 1) 단지코드별 총면적 합 집계

- 단지코드별 총면적 합을 집계합니다.
- 집계한 결과를 df_area 데이터프레임으로 선언합니다.

In [428]:
df_area = data02.groupby('단지코드')[['총면적']].sum()
df_area.head()

,총면적
단지코드,
C0001,6023.7683
C0002,1569.1668
C0003,7180.1396
C0004,47058.9273
C0006,543.0268


#### 2) 전용면적 구간별 집계 (피벗 형태)

- data02 데이터프레임에 전용면적을 몇몇 구간으로 나눈 범줏값을 갖는 변수를 추가합니다.
- 구간을 어떻게 나눌 지 충분히 고민해 봅니다.
    - 구간 예: 10-30, 30-40, 40-50, 50-60, 60-70, 70-80, 80-200 
- 추가할 변수 이름은 '전용면적구간'으로 합니다.
- 참고: pd.cut() 함수를 활용합니다.

In [430]:
data02.describe()

,총면적,전용면적별세대수,전용면적,공용면적,임대보증금,임대료
count,1157.000000,1157.000000,1157.000000,1157.000000,1.157000e+03,1.157000e+03
mean,11117.129107,163.691443,51.565584,20.562360,2.850789e+07,2.259409e+05
std,11417.523851,166.766358,18.243315,5.164405,2.890687e+07,1.768102e+05
min,30.095500,1.000000,17.590000,5.850000,0.000000e+00,0.000000e+00
25%,3070.670200,44.000000,39.480000,16.997400,1.379700e+07,1.177400e+05
50%,7734.663000,112.000000,46.900000,20.384700,1.997300e+07,1.842900e+05
75%,15042.642300,229.000000,59.810000,23.722500,3.375300e+07,2.634400e+05
max,90472.674100,1258.000000,139.350000,42.760000,2.549220e+08,1.058030e+06


In [431]:
data02['전용면적구간'] = pd.cut(data02['전용면적'], bins =[10, 30, 40, 50, 60, 70, 80, np.inf], labels = ['면적10_30', '면적30_40', '면적40_50', '면적50_60', '면적60_70', '면적70_80', '면적80_200'])
data02.head()

,단지코드,총면적,전용면적별세대수,전용면적,공용면적,임대보증금,임대료,전용면적구간
0,C0001,2490.2605,35,51.89,19.2603,50758000,620370,면적50_60
1,C0001,3533.5078,43,59.93,22.2446,63166000,665490,면적50_60
2,C0002,1151.4750,26,27.75,16.5375,63062000,458640,면적10_30
3,C0002,417.6918,9,29.08,17.3302,63062000,481560,면적10_30
4,C0003,569.9134,7,59.47,21.9462,72190000,586540,면적50_60


- 단지코드, 전용면적구간별 전용면적별세대수 합을 집계합니다.
- 집계 결과를 temp 데이터프레임으로 선언합니다.

In [433]:
temp = data02.groupby(['단지코드','전용면적구간'])[['전용면적별세대수']].sum().reset_index()
temp.head()

,단지코드,전용면적구간,전용면적별세대수
0,C0001,면적10_30,0
1,C0001,면적30_40,0
2,C0001,면적40_50,0
3,C0001,면적50_60,78
4,C0001,면적60_70,0


- temp 데이터프레임을 피벗 형태로 변환하여 df_pivot 데이터프레임으로 선언합니다.
- 인덱스를 초기화합니다. (단, 인덱스였던 '단지코드'가 제거되면 안됨)
- 이후 작업의 편의를 위해 일반적인 데이터프레임 형태를 갖게 합니다.
- 참고: df2 = df1.pivot(index=?, columns=?, values=?) 형태로 pivot() 메서드를 사용합니다.
- 참고: df2.columns.name=None 형태의 구문을 사용해 열이름에 대한 이름을 제거합니다.

In [502]:
df_pivot = temp.pivot(index='단지코드', columns='전용면적구간', values='전용면적별세대수').reset_index()
df_pivot.columns.name=None
df_pivot.head()

,단지코드,면적10_30,면적30_40,면적40_50,면적50_60,면적60_70,면적70_80,면적80_200
0,C0001,0,0,0,78,0,0,0
1,C0002,35,0,0,0,0,0,0
2,C0003,0,0,0,88,0,0,0
3,C0004,0,0,0,150,0,216,111
4,C0006,15,0,0,0,0,0,0


#### 3) 임대보증금, 임대료 평균 집계

- 단지코드별 임대보증금, 임대료 평균을 집계합니다.
- 집계 결과를 df_rent 데이터프레임으로 선언합니다.

In [482]:
df_rent = data02.groupby(['단지코드'])[['임대보증금','임대료']].mean()
df_rent.head()

,임대보증금,임대료
단지코드,,
C0001,5.696200e+07,642930.000000
C0002,6.306200e+07,470100.000000
C0003,7.219000e+07,586540.000000
C0004,1.015167e+08,950305.000000
C0006,5.522750e+07,340148.333333


In [497]:
# 전용면적별 세대수를 반영하여 mean구하기
df_rent1 = data02.groupby('단지코드').apply( lambda x: pd.Series({
        '임대보증금': (x['임대보증금'] * x['전용면적별세대수']).sum() / x['전용면적별세대수'].sum(),
        '임대료': (x['임대료'] * x['전용면적별세대수']).sum() / x['전용면적별세대수'].sum()
    })
).reset_index()

df_rent1.head()

,단지코드,임대보증금,임대료
0,C0001,5.759831e+07,645243.846154
1,C0002,6.306200e+07,464533.714286
2,C0003,7.219000e+07,586540.000000
3,C0004,9.158473e+07,877880.272537
4,C0006,5.465420e+07,334797.333333


### (3) 집계 결과 합치기

- 위 과정에서 만든 df_area, df_pivot, df_rent 데이터프레임을 data01 데이터프레임과 조인(Merge)합니다.
- data01 데이터프레임이 기준 데이터프레임입니다.
- '단지코드' 변수가 조인 기준이 되며, how='left'를 지정합니다.
- 조인 결과를 base_data 데이터프레임으로 선언합니다.

3.3m^2 = 1평

In [491]:
base_data = pd.merge(df_area, df_pivot, how = 'left',on = '단지코드')
base_data = pd.merge(base_data, df_rent, how = 'left',on = '단지코드')
base_data = pd.merge(data01, base_data, how = 'left', on = '단지코드')
base_data

,단지코드,총세대수,지역,준공연도,건물형태,난방방식,승강기설치여부,실차량수,총면적,면적10_30,면적30_40,면적40_50,면적50_60,면적60_70,면적70_80,면적80_200,임대보증금,임대료
0,C0001,78,서울,2013,계단식,개별가스난방,전체동 설치,109,6023.7683,0,0,0,78,0,0,0,5.696200e+07,642930.000000
1,C0002,35,서울,2013,복도식,개별가스난방,전체동 설치,35,1569.1668,35,0,0,0,0,0,0,6.306200e+07,470100.000000
2,C0003,88,서울,2013,계단식,개별가스난방,전체동 설치,88,7180.1396,0,0,0,88,0,0,0,7.219000e+07,586540.000000
3,C0004,477,서울,2014,복도식,지역난방,전체동 설치,943,47058.9273,0,0,0,150,0,216,111,1.015167e+08,950305.000000
4,C0006,15,서울,2013,복도식,개별가스난방,전체동 설치,21,543.0268,15,0,0,0,0,0,0,5.522750e+07,340148.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,C0354,1485,대전충남,1993,복도식,중앙난방,전체동 설치,298,64622.2500,1181,298,0,6,0,0,0,7.595571e+06,104975.714286
341,C1354,1386,대전충남,1993,복도식,중앙가스난방,전체동 설치,258,57616.8100,1071,298,0,17,0,0,0,8.092875e+06,111848.750000
342,C0356,956,경기,1994,복도식,지역가스난방,전체동 설치,243,37398.7200,956,0,0,0,0,0,0,9.931000e+06,134540.000000
343,C0358,120,강원,2020,복도식,개별가스난방,전체동 설치,47,5581.8024,66,54,0,0,0,0,0,2.515500e+06,50040.000000


In [525]:
#base_data.drop(columns = base_data[['총세대수','실차량수']].loc[(base_data['총세대수']<=100)&(base_data['실차량수']-base_data['총세대수']*2>0).reset_index()],inplace=True)

## 4. 데이터 셋 저장

- joblib.dump() 함수를 사용하여 최종 데이터 셋을 파일로 저장합니다.
- 파일 이름은 base_data1.pkl로 합니다.

In [499]:
# 파일로 저장
joblib.dump(base_data, path+'base_data1.pkl')

['base_data1.pkl']